# Debarment Data Extraction

In [9]:
import tabula
import pandas as pd
import os
import re

In [2]:
# pdf path names
## can also read from raw dir if 
## path names change
path1 = "https://www.foreignlaborcert.doleta.gov/pdf/H-2A_Expired_Debarments_OFLC_webpage_Final_Draft.pdf"
path2 = "https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/Debarment_List.pdf"

### Pulling out h2a debarment data

In [3]:
# read in with tabula
debar_h2a = tabula.read_pdf(path1, lattice=True, pages = "all", pandas_options={'header':None})
# pull out headers
headers = debar_h2a[0].values.tolist()[2]
# cut out excessive rows at the beginning
debar_h2a[0] = debar_h2a[0].iloc[5:]
# assign headers to each df in the list of dfs
for df in debar_h2a: df.columns = headers
# concat all df
h2a_concat = pd.concat(debar_h2a)

In [4]:
pd.set_option('display.max_rows', 200)


In [17]:
# deal with Nas
h2a_concat = h2a_concat[~h2a_concat.Name.str.contains('FY', na=False)]
h2a_final = h2a_concat.dropna(how='all').copy()
h2a_final.reset_index(drop=True, inplace=True)

In [18]:
h2a_final.isnull().sum()

print(h2a_final.head())

## remove linebreaks from name and violation col which messes up csv writing
nolb_name = [re.sub("\\r", " ", s) for s in h2a_final.Name]
nolb_viol = [re.sub("\\r", " ", s) for s in h2a_final.Violation]
h2a_final['Name'] = nolb_name
h2a_final['Violation'] = nolb_viol
h2a_final.head()

                                Name     City, State  \
0                     J&J Harvesting       Leads, ND   
1             Stahlman Apiaries, Inc       Selby, SD   
2                      Trust Nursery     Pulaski, NY   
3              Anton Fertilizer Inc.     Dighton, KS   
4  Great Plains Fluid Service,\rInc.  Greensburg, KS   

                                         Violation Duration Start date  \
0  Failure to respond to audit (partial\rresponse)  2 years  1/19/2014   
1  Failure to respond to audit (partial\rresponse)   1 year  2/19/2015   
2  Failure to respond to audit (partial\rresponse)   1 year  3/21/2014   
3       Failure to respond to audit (no\rresponse)  2 years  3/30/2014   
4       Failure to respond to audit (no\rresponse)  2 years  3/30/2014   

    End date  
0  1/18/2016  
1  2/14/2016  
2  3/20/2015  
3  3/29/2016  
4  3/29/2016  


,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


### Pulling out debarment data

In [20]:
# read in with tabula
debar = tabula.read_pdf(path2, lattice=True, pages = "all", pandas_options={'header':None})
# pull out headers
headers2 = debar[0].values.tolist()[2]
# cut out excessive rows at the beginning
for i in range(0,6):
    debar[i] = debar[i].iloc[4:]
#debar[0] = debar[0].iloc[4:]
# assign headers to each df in the list of dfs
for df in debar: df.columns = headers2
# concat all df
debar_concat = pd.concat(debar)
# cleaning and index
debar_final = debar_concat.replace('\r',' ', regex=True)
debar_final = debar_final.dropna(how='all') 
debar_final.reset_index(drop=True, inplace=True)

In [22]:
debar_final.isnull().sum()

debar_final.head()

,Entity,Entity Type,Employer Location,Start of Debarment,End of Debarment,Violation,CFR Citation
0,Grace Yu,Agent,"Duluth, Georgia","May 13, 2020","May 12, 2023",Failure to respond to audit request,20 CFR §§ 656.20 and 656.31(f)(1)(iv)
1,"Harrison Poultry, Inc.",Employer,"Bethlehem, Georgia","June 29, 2018","June 29, 2021",Failure to respond to audit request,20 CFR §§ 656.20 and 656.31(f)(1)(iv)
2,Raul G. Sebazco,Agent,"Miami, Florida","September 2, 2020","September 1, 2023",Participated or facilitated prohibited actions,20 CFR 656.31(f)(1)(i)
3,"Lorenzo Construction, LLC",Employer,"Crofton, Maryland","February 26, 2021","February 26, 2022",Failure to respond to audit request,20 CFR §§ 656.20 and 656.31(f)(1)(iv)
4,"Victory Processing, LLC",Employer,"Gainesville, Georgia","March 31, 2021","March 21, 2022",Failure to respond to audit request,20 CFR §§ 656.20 and 656.31(f)(1)(iv)


In [23]:
### look at overlap with other debarment file
debar_ent_cap = debar_final.Entity.str.upper()
debar_name_cap = h2a_final.Name.str.upper()

### no intersect at least with partially cleaned entities
len(set(debar_ent_cap).intersection(debar_name_cap))

0

In [24]:
# save to csv
WRITE_CSV = True
if WRITE_CSV:
    debar_final.to_csv("../data/intermediate/current_debarment.csv", index = False)
    h2a_final.to_csv("../data/intermediate/expired_h2a_debarment.csv", index = False)
